<!-- <a href="https://colab.research.google.com/github/i1idan/schizophrenia-diagnosis-eeg-signals/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> -->

# Install Requirements

In [26]:
!pip install -r requirements.txt

     |████████████████████████████████| 67 kB 51 kB/s eta 0:00:01


# Get Data

In [5]:
# 2016
dataset_name = "ISIC_2016"
train_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Training_Data.zip"
test_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Test_Data.zip"
mask_train_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Training_GroundTruth.zip"
mask_test_zip_url = "https://isic-challenge-data.s3.amazonaws.com/2016/ISBI2016_ISIC_Part1_Test_GroundTruth.zip"
train_path = "data/train"
test_path = "data/test"
mask_train_path = "data/mask_train"
mask_test_path = "data/mask_test"

In [10]:
zip_name = train_zip_url.split("/")[-1]

import os
os.path.isfile(zip_name)

True

In [27]:
!python data/preprocessing.py \
--dataset_name $dataset_name \
--train_zip_url $train_zip_url \
--test_zip_url $test_zip_url \
--mask_train_zip_url $mask_train_zip_url \
--mask_test_zip_url $mask_test_zip_url \
--train_path $train_path \
--test_path $test_path \
--mask_train_path $mask_train_path \
--mask_test_path $mask_test_path

/home/ai/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/deep_utils/vision/face_detection/ultralight/__init__.py:3: UserWarning: 
No module named 'deep_utils.vision.face_detection.ultralight.torch'. If you don't use UltralightTorchFaceDetector ignore this message.
  UltralightTorchFaceDetector = import_module(
/home/ai/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/deep_utils/vision/face_detection/ultralight/__init__.py:6: UserWarning: 
No module named 'deep_utils.vision.face_detection.ultralight.tf'. If you don't use UltralightTFFaceDetector ignore this message.
  UltralightTFFaceDetector = import_module(
[INFO] zip file: ISBI2016_ISIC_Part1_Training_Data doesn't exist! Unzipping ISBI2016_ISIC_Part1_Training_Data.zip
[INFO] Successfully extracted ISBI2016_ISIC_Part1_Training_Data.zip to .!
[INFO] zip file: ISBI2016_ISIC_Part1_Training_GroundTruth doesn't exist! Unzipping ISBI2016_ISIC_Part1_Training_GroundTruth.zip
[INFO] Successfully extracted ISBI2016_ISIC_Part1_Training_Grou

# Training Models
## UnetRes50
### Main params

In [77]:
#basic
dataset_name = "ISIC_2016"
model = 'unet_res50'
save_path = "saved_models/"
train_identifier = "usual"
multi_train = 3
epochs = 50
batch_size = 32
img_size = (256, 256)
img_channel = 3
loss = "dice_loss"

# optimizer
optimizer = 'adam'

# learning rate
lr = 1e-3
min_lr = 1e-5

# callbacks
reduce_lr_patience = 10
reduce_lr_factor = 0.1
early_stopping_p = 25

seeds = 1234
verbose = 1

# Augmentation
hair_aug_p = 0
hair_rmv_p = 0
random_rotate_p = 0.5
p_horizontal_flip = 0.5
p_vertical_flip = 0.5
p_center_crop = 0.5
mosaic_p = 0
cutmix_p = 0
cutmix_beta = 0

In [78]:
!PYTHONHASHSEED=0
!TF_DETERMINISTIC_OPS=0
!TF_CUDNN_DETERMINISTIC=0
for n in range(multi_train):
    seed = seeds + n
    saved_path_name = f"{train_identifier}_{n}"
    !python train.py \
    --model $model --dataset_name $dataset_name \
    --save_path $save_path \
    --save_path_name $saved_path_name \
    --epochs $epochs \
    --optimizer $optimizer \
    --lr $lr \
    --min_lr $min_lr \
    --reduce_lr_patience $reduce_lr_patience \
    --reduce_lr_factor $reduce_lr_factor \
    --early_stopping_p $early_stopping_p \
    --hair_aug_p $hair_aug_p \
    --hair_rmv_p $hair_rmv_p \
    --random_rotate_p $random_rotate_p \
    --p_horizontal_flip $p_horizontal_flip \
    --p_vertical_flip $p_vertical_flip \
    --p_center_crop $p_center_crop \
    --mosaic_p $mosaic_p \
    --cutmix_p $cutmix_p \
    --cutmix_beta $cutmix_beta \
    --seed $seed
    
    print(f"\n-----------------------------train {n} is done! ----------------------------------\n")

/home/ai/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/deep_utils/vision/face_detection/ultralight/__init__.py:3: UserWarning: 
No module named 'deep_utils.vision.face_detection.ultralight.torch'. If you don't use UltralightTorchFaceDetector ignore this message.
  UltralightTorchFaceDetector = import_module(
/home/ai/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/deep_utils/vision/face_detection/ultralight/__init__.py:6: UserWarning: 
No module named 'deep_utils.vision.face_detection.ultralight.tf'. If you don't use UltralightTFFaceDetector ignore this message.
  UltralightTFFaceDetector = import_module(
2022-02-26 22:02:16.313099: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26 22:02:16.330304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there

2022-02-26 22:02:17.356286: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-02-26 22:02:17.356444: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
[INFO] Successfully created following callbacks [model_checkpoint, reduce_lr, early_stop, csv_logger, tensorboard]
2022-02-26 22:02:17,359 - SKIN-CANCER - INFO - loss: dice_loss is successfully created!
2022-02-26 22:02:17,363 - SKIN-CANCER - INFO - Start train for saved_models/ISIC_2016/unet_res50/usual_0/exp_2
Epoch 1/50
2022-02-26 22:02:20.300097: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8301
 1/90 [..............................] - ETA: 6:59 - loss: 0.6835 - dice_score: 0.3165 - iou: 0.1880 - dice_loss: 0.6835 - jaccard_loss: 0.7197 - focal_tversky_loss: 0.68972022-02-26 22:02:22.323203: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-02-26 22:02:22.323223: I tensorflow/core/profi

90/90 [==============================] - ETA: 0s - loss: 0.1086 - dice_score: 0.8914 - iou: 0.8062 - dice_loss: 0.1086 - jaccard_loss: 0.1891 - focal_tversky_loss: 0.1968
Epoch 00009: val_loss did not improve from 0.48851
90/90 [==============================] - 28s 308ms/step - loss: 0.1086 - dice_score: 0.8914 - iou: 0.8062 - dice_loss: 0.1086 - jaccard_loss: 0.1891 - focal_tversky_loss: 0.1968 - val_loss: 0.5561 - val_dice_score: 0.4439 - val_iou: 0.2976 - val_dice_loss: 0.5561 - val_jaccard_loss: 0.6999 - val_focal_tversky_loss: 0.7005 - lr: 1.0000e-04
Epoch 10/50
90/90 [==============================] - ETA: 0s - loss: 0.1017 - dice_score: 0.8983 - iou: 0.8175 - dice_loss: 0.1017 - jaccard_loss: 0.1780 - focal_tversky_loss: 0.1813
Epoch 00010: val_loss improved from 0.48851 to 0.23403, saving model to saved_models/ISIC_2016/unet_res50/usual_0/exp_2/model
90/90 [==============================] - 34s 383ms/step - loss: 0.1017 - dice_score: 0.8983 - iou: 0.8175 - dice_loss: 0.1017 - 

90/90 [==============================] - ETA: 0s - loss: 0.0736 - dice_score: 0.9264 - iou: 0.8636 - dice_loss: 0.0736 - jaccard_loss: 0.1322 - focal_tversky_loss: 0.1431
Epoch 00023: val_loss did not improve from 0.09623
90/90 [==============================] - 27s 303ms/step - loss: 0.0736 - dice_score: 0.9264 - iou: 0.8636 - dice_loss: 0.0736 - jaccard_loss: 0.1322 - focal_tversky_loss: 0.1431 - val_loss: 0.0980 - val_dice_score: 0.9020 - val_iou: 0.8262 - val_dice_loss: 0.0980 - val_jaccard_loss: 0.1698 - val_focal_tversky_loss: 0.1684 - lr: 1.0000e-04
Epoch 24/50
90/90 [==============================] - ETA: 0s - loss: 0.0706 - dice_score: 0.9294 - iou: 0.8689 - dice_loss: 0.0706 - jaccard_loss: 0.1272 - focal_tversky_loss: 0.1369
Epoch 00024: val_loss did not improve from 0.09623

Epoch 00024: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
90/90 [==============================] - 28s 307ms/step - loss: 0.0706 - dice_score: 0.9294 - iou: 0.8689 - dice_loss: 0.

90/90 [==============================] - ETA: 0s - loss: 0.0675 - dice_score: 0.9325 - iou: 0.8742 - dice_loss: 0.0675 - jaccard_loss: 0.1217 - focal_tversky_loss: 0.1337
Epoch 00037: val_loss did not improve from 0.09623
90/90 [==============================] - 27s 300ms/step - loss: 0.0675 - dice_score: 0.9325 - iou: 0.8742 - dice_loss: 0.0675 - jaccard_loss: 0.1217 - focal_tversky_loss: 0.1337 - val_loss: 0.1154 - val_dice_score: 0.8846 - val_iou: 0.8048 - val_dice_loss: 0.1154 - val_jaccard_loss: 0.1916 - val_focal_tversky_loss: 0.1872 - lr: 1.0000e-05
Epoch 38/50
90/90 [==============================] - ETA: 0s - loss: 0.0656 - dice_score: 0.9344 - iou: 0.8775 - dice_loss: 0.0656 - jaccard_loss: 0.1185 - focal_tversky_loss: 0.1289
Epoch 00038: val_loss did not improve from 0.09623
90/90 [==============================] - 27s 304ms/step - loss: 0.0656 - dice_score: 0.9344 - iou: 0.8775 - dice_loss: 0.0656 - jaccard_loss: 0.1185 - focal_tversky_loss: 0.1289 - val_loss: 0.1017 - val_

...

2022-02-26 22:45:57.286740: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-02-26 22:45:57.287093: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
[INFO] Successfully created following callbacks [model_checkpoint, reduce_lr, early_stop, csv_logger, tensorboard]
2022-02-26 22:45:57,289 - SKIN-CANCER - INFO - loss: dice_loss is successfully created!
2022-02-26 22:45:57,293 - SKIN-CANCER - INFO - Start train for saved_models/ISIC_2016/unet_res50/usual_2/exp_2
Epoch 1/50
2022-02-26 22:46:00.522873: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8301
 1/90 [..............................] - ETA: 7:39 - loss: 0.6676 - dice_score: 0.3324 - iou: 0.1993 - dice_loss: 0.6676 - jaccard_loss: 0.6954 - focal_tversky_loss: 0.68872022-02-26 22:46:02.782549: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-02-26 22:46:02.782571: I tensorflow/core/profi

90/90 [==============================] - 33s 370ms/step - loss: 0.1047 - dice_score: 0.8953 - iou: 0.8131 - dice_loss: 0.1047 - jaccard_loss: 0.1826 - focal_tversky_loss: 0.1859 - val_loss: 0.9175 - val_dice_score: 0.0825 - val_iou: 0.0438 - val_dice_loss: 0.9175 - val_jaccard_loss: 0.9561 - val_focal_tversky_loss: 0.9535 - lr: 0.0010
Epoch 9/50
90/90 [==============================] - ETA: 0s - loss: 0.1210 - dice_score: 0.8790 - iou: 0.7868 - dice_loss: 0.1210 - jaccard_loss: 0.2092 - focal_tversky_loss: 0.2032
Epoch 00009: val_loss improved from 0.91754 to 0.61587, saving model to saved_models/ISIC_2016/unet_res50/usual_2/exp_2/model
90/90 [==============================] - 33s 369ms/step - loss: 0.1210 - dice_score: 0.8790 - iou: 0.7868 - dice_loss: 0.1210 - jaccard_loss: 0.2092 - focal_tversky_loss: 0.2032 - val_loss: 0.6159 - val_dice_score: 0.3841 - val_iou: 0.2556 - val_dice_loss: 0.6159 - val_jaccard_loss: 0.7432 - val_focal_tversky_loss: 0.7401 - lr: 0.0010
Epoch 10/50
90/90 

90/90 [==============================] - ETA: 0s - loss: 0.0985 - dice_score: 0.9015 - iou: 0.8231 - dice_loss: 0.0985 - jaccard_loss: 0.1743 - focal_tversky_loss: 0.1742
Epoch 00022: val_loss did not improve from 0.11443

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
90/90 [==============================] - 28s 309ms/step - loss: 0.0985 - dice_score: 0.9015 - iou: 0.8231 - dice_loss: 0.0985 - jaccard_loss: 0.1743 - focal_tversky_loss: 0.1742 - val_loss: 0.1601 - val_dice_score: 0.8399 - val_iou: 0.7300 - val_dice_loss: 0.1601 - val_jaccard_loss: 0.2680 - val_focal_tversky_loss: 0.2263 - lr: 0.0010
Epoch 23/50
90/90 [==============================] - ETA: 0s - loss: 0.0827 - dice_score: 0.9173 - iou: 0.8480 - dice_loss: 0.0827 - jaccard_loss: 0.1496 - focal_tversky_loss: 0.1541
Epoch 00023: val_loss improved from 0.11443 to 0.11104, saving model to saved_models/ISIC_2016/unet_res50/usual_2/exp_2/model
90/90 [==============================] - 33s 373ms

90/90 [==============================] - ETA: 0s - loss: 0.0670 - dice_score: 0.9330 - iou: 0.8751 - dice_loss: 0.0670 - jaccard_loss: 0.1226 - focal_tversky_loss: 0.1317
Epoch 00036: val_loss did not improve from 0.09205
90/90 [==============================] - 27s 301ms/step - loss: 0.0670 - dice_score: 0.9330 - iou: 0.8751 - dice_loss: 0.0670 - jaccard_loss: 0.1226 - focal_tversky_loss: 0.1317 - val_loss: 0.1036 - val_dice_score: 0.8964 - val_iou: 0.8222 - val_dice_loss: 0.1036 - val_jaccard_loss: 0.1756 - val_focal_tversky_loss: 0.1835 - lr: 1.0000e-04
Epoch 37/50
90/90 [==============================] - ETA: 0s - loss: 0.0692 - dice_score: 0.9308 - iou: 0.8714 - dice_loss: 0.0692 - jaccard_loss: 0.1263 - focal_tversky_loss: 0.1355
Epoch 00037: val_loss did not improve from 0.09205
90/90 [==============================] - 27s 301ms/step - loss: 0.0692 - dice_score: 0.9308 - iou: 0.8714 - dice_loss: 0.0692 - jaccard_loss: 0.1263 - focal_tversky_loss: 0.1355 - val_loss: 0.0987 - val_

90/90 [==============================] - ETA: 0s - loss: 0.0652 - dice_score: 0.9348 - iou: 0.8781 - dice_loss: 0.0652 - jaccard_loss: 0.1195 - focal_tversky_loss: 0.1293
Epoch 00050: val_loss did not improve from 0.09205
90/90 [==============================] - 27s 303ms/step - loss: 0.0652 - dice_score: 0.9348 - iou: 0.8781 - dice_loss: 0.0652 - jaccard_loss: 0.1195 - focal_tversky_loss: 0.1293 - val_loss: 0.1103 - val_dice_score: 0.8897 - val_iou: 0.8158 - val_dice_loss: 0.1103 - val_jaccard_loss: 0.1822 - val_focal_tversky_loss: 0.1868 - lr: 1.0000e-05
2022-02-26 23:10:18,347 - SKIN-CANCER - INFO - Training is done model is saved in saved_models/ISIC_2016/unet_res50/usual_2/exp_2
Evaluation
48/48 [==============================] - 8s 174ms/step - loss: 0.1048 - dice_score: 0.8952 - iou: 0.8261 - dice_loss: 0.1048 - jaccard_loss: 0.1718 - focal_tversky_loss: 0.1702
2022-02-26 23:10:26,734 - SKIN-CANCER - INFO - Test: Loss= 0.10476084798574448, Dice-Score: 0.8952392935752869, IoU: 0.

# Get Metrics

In [85]:
from utils.group_metrics import get_mean_std
import os

csv_addresses = [
 os.path.join(save_path, dataset_name, model, f"{train_identifier}_{n}", "exp_2" , "csv_logger_train.csv") for n in range(multi_train)   
]
metrics = get_mean_std(csv_addresses, 
                       arguments=(
                           "dice_loss",
                           "dice_score",
                           "focal_tversky_loss",
                           "iou",
                           "jaccard_loss",
                           "loss",
                           "val_dice_loss",
                           "val_dice_score",
                           "val_focal_tversky_loss",
                           "val_iou",
                           "val_jaccard_loss",
                           "val_loss"
                       ),
                       operators=(min, max, min, max, min, min, min, max, min, max, min, min)
                      )
print(metrics)

[INFO] Extracting values from csv files...
[INFO] Getting the values of file saved_models/ISIC_2016/unet_res50/usual_0/exp_2/csv_logger_train.csv
[INFO] Getting the values of file saved_models/ISIC_2016/unet_res50/usual_1/exp_2/csv_logger_train.csv
[INFO] Getting the values of file saved_models/ISIC_2016/unet_res50/usual_2/exp_2/csv_logger_train.csv
{'dice_loss': {'std': 0.0041, 'mean': 0.0677}, 'dice_score': {'std': 0.0041, 'mean': 0.9323}, 'focal_tversky_loss': {'std': 0.0062, 'mean': 0.1324}, 'iou': {'std': 0.007, 'mean': 0.8739}, 'jaccard_loss': {'std': 0.0063, 'mean': 0.1225}, 'loss': {'std': 0.0041, 'mean': 0.0677}, 'val_dice_loss': {'std': 0.0055, 'mean': 0.0904}, 'val_dice_score': {'std': 0.0055, 'mean': 0.9096}, 'val_focal_tversky_loss': {'std': 0.0067, 'mean': 0.1612}, 'val_iou': {'std': 0.0085, 'mean': 0.8366}, 'val_jaccard_loss': {'std': 0.0088, 'mean': 0.1597}, 'val_loss': {'std': 0.0055, 'mean': 0.0904}}


In [83]:
csv_addresses

['ISIC_2016/unet_res50/usual_0/exp_2/csv_logger_train.csv',
 'ISIC_2016/unet_res50/usual_1/exp_2/csv_logger_train.csv',
 'ISIC_2016/unet_res50/usual_2/exp_2/csv_logger_train.csv']

_:)_